In [1]:
import warnings
warnings.filterwarnings("ignore")
import tensorflow as tf
import matplotlib.pyplot as plt
tf.compat.v1.set_random_seed(0)
from tensorflow import keras
import numpy as np
np.random.seed(0)
import itertools
from keras.preprocessing.image import image_dataset_from_directory
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from sklearn.metrics import precision_score, accuracy_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

<h1>Data Loading</h1>
<p>Setting up Image Data Generators<p>

In [2]:
#train_gen = image_dataset_from_directory(directory="../input/new-plant-diseases-dataset/train",image_size=(256, 256))
#test_gen = image_dataset_from_directory(directory="../input/new-plant-diseases-dataset/valid",image_size=(256, 256))

train_gen = image_dataset_from_directory(directory="../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train",
                                         image_size=(256, 256))
test_gen = image_dataset_from_directory(directory="../input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid",
                                        image_size=(256, 256))

rescale = Rescaling(scale=1.0/255)
train_gen = train_gen.map(lambda image,label:(rescale(image),label))
test_gen  = test_gen.map(lambda image,label:(rescale(image),label))

Found 70295 files belonging to 38 classes.
Found 17572 files belonging to 38 classes.


<h1>Modelling</h1>

In [9]:
from tensorflow import keras
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models

# Load pretrained VGG16 as the base (exclude the fully connected top layers)
base_model = VGG16(
    weights='imagenet',        # load ImageNet pretrained weights
    include_top=False,         # drop the FC layers at the top
    input_shape=(256, 256, 3)  # your input shape
)

# Freeze the base model (so we don’t retrain ImageNet weights initially)
base_model.trainable = False

# Build the model
model = models.Sequential([
    base_model,                          # feature extractor
    layers.Flatten(),                    # flatten the conv feature maps
    layers.Dense(512, activation='relu'),# custom dense layer
    layers.Dropout(0.5),
    layers.Dense(38, activation='softmax')  # 38 classes output
])

# Compile
opt = keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


58900480/58889256 [==============================] - 0s 0us/step
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 8, 8, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               16777728  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 38)                19494     
Total params: 31,511,910
Trainable params: 16,797,222
Non-trainable params: 14,714,688
_________________________________________________________________


In [10]:
ep = 10
history = model.fit_generator(train_gen,
          validation_data=test_gen,
          epochs = ep)

Epoch 1/10
2197/2197 [==============================] - 219s 98ms/step - loss: 0.8286 - accuracy: 0.7645 - val_loss: 0.2611 - val_accuracy: 0.9280
Epoch 2/10
2197/2197 [==============================] - 213s 97ms/step - loss: 0.3319 - accuracy: 0.8982 - val_loss: 0.1863 - val_accuracy: 0.9426
Epoch 3/10
2197/2197 [==============================] - 218s 99ms/step - loss: 0.2354 - accuracy: 0.9257 - val_loss: 0.1506 - val_accuracy: 0.9533
Epoch 4/10
2197/2197 [==============================] - 212s 96ms/step - loss: 0.1840 - accuracy: 0.9409 - val_loss: 0.1299 - val_accuracy: 0.9578
Epoch 5/10
2197/2197 [==============================] - 212s 96ms/step - loss: 0.1487 - accuracy: 0.9519 - val_loss: 0.1292 - val_accuracy: 0.9569
Epoch 6/10
2197/2197 [==============================] - 212s 96ms/step - loss: 0.1287 - accuracy: 0.9578 - val_loss: 0.1116 - val_accuracy: 0.9628
Epoch 7/10
2197/2197 [==============================] - 212s 97ms/step - loss: 0.1072 - accuracy: 0.9646 - val_loss: 0

<h1>Metrics</h1>

In [11]:
labels = []
predictions = []
for x,y in test_gen:
    labels.append(list(y.numpy()))
    predictions.append(tf.argmax(model.predict(x),1).numpy())

In [12]:
predictions = list(itertools.chain.from_iterable(predictions))
labels = list(itertools.chain.from_iterable(labels))

In [13]:
print("Train Accuracy  : {:.2f} %".format(history.history['accuracy'][-1]*100))
print("Test Accuracy   : {:.2f} %".format(accuracy_score(labels, predictions) * 100))
print("Precision Score : {:.2f} %".format(precision_score(labels, predictions, average='micro') * 100))
print("Recall Score    : {:.2f} %".format(recall_score(labels, predictions, average='micro') * 100))

Train Accuracy  : 97.60 %
Test Accuracy   : 96.63 %
Precision Score : 96.63 %
Recall Score    : 96.63 %
